In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import itertools 
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import pandas as pd
from tqdm.notebook import tqdm


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from IPython import display
from jiwer import wer

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# 음성파일 현재 경로에 UnZip 하기

In [ ]:
%cd "/content"

/content


In [ ]:
pwd

'/content'

In [ ]:
#https://drive.google.com/file/d/1L7_EMfeuUx6Di6qYg5Qgu7W2MHgxpELb/view?usp=sharing
!gdown --id 1L7_EMfeuUx6Di6qYg5Qgu7W2MHgxpELb

In [ ]:
!unzip -q /content/drive/MyDrive/jeju_stand.zip

# json 파일 읽고 필요한 정보 뽑기

In [ ]:
# import json

In [ ]:
# with open('/content/jeju_stand/DZJD21002362.json') as f:
#   data = json.load(f)

In [ ]:
# data

# 음성데이터 자르기

In [ ]:
df_la = pd.read_csv("/content/drive/MyDrive/jeju_to_stand_label.csv")

In [ ]:
df_la

,Unnamed: 0,name,start,end,standard_form,dialect_form
0,0,DZJD21002394,1.50,3.39,저는 &name& 입니다.,저는 &name& 입니다.
1,1,DZJD21002394,4.75,6.64,저는 &name& 입니다.,저는 &name& 입니다.
2,2,DZJD21002394,9.25,12.10,시작하세요 대화 먼저,시작하세요 대화 먼저
3,3,DZJD21002394,12.75,20.17,어 편하게 역사 그러면 하자 뭐 역사 어쨌든 얘기했던 거니까.,어 편하게 역사 그러면 하주 뭐 역사 어쨌든 얘기했던 거니까.
4,4,DZJD21002394,20.75,21.83,뭐하지,뭐하지
...,...,...,...,...,...,...
2255,2255,DZJD21002363,1683.84,1684.83,비싸더라,비싸더라
2256,2256,DZJD21002363,1685.25,1686.87,그런 점이 싫어졌어,그런 점이 싫어졌어
2257,2257,DZJD21002363,1689.50,1692.47,집에만 있으면 나오는게 그거 였는데,집에만 있으면 나오는게 그거 여신디
2258,2258,DZJD21002363,1695.00,1696.89,집에 가도 안 나와요,집에 가도 안 나와요


In [ ]:
df_la["dura"] = df_la["end"] - df_la["start"]

In [ ]:
df_la

,Unnamed: 0,name,start,end,standard_form,dialect_form,dura
0,0,DZJD21002394,1.50,3.39,저는 &name& 입니다.,저는 &name& 입니다.,1.89
1,1,DZJD21002394,4.75,6.64,저는 &name& 입니다.,저는 &name& 입니다.,1.89
2,2,DZJD21002394,9.25,12.10,시작하세요 대화 먼저,시작하세요 대화 먼저,2.85
3,3,DZJD21002394,12.75,20.17,어 편하게 역사 그러면 하자 뭐 역사 어쨌든 얘기했던 거니까.,어 편하게 역사 그러면 하주 뭐 역사 어쨌든 얘기했던 거니까.,7.42
4,4,DZJD21002394,20.75,21.83,뭐하지,뭐하지,1.08
...,...,...,...,...,...,...,...
2255,2255,DZJD21002363,1683.84,1684.83,비싸더라,비싸더라,0.99
2256,2256,DZJD21002363,1685.25,1686.87,그런 점이 싫어졌어,그런 점이 싫어졌어,1.62
2257,2257,DZJD21002363,1689.50,1692.47,집에만 있으면 나오는게 그거 였는데,집에만 있으면 나오는게 그거 여신디,2.97
2258,2258,DZJD21002363,1695.00,1696.89,집에 가도 안 나와요,집에 가도 안 나와요,1.89


In [ ]:
file_name = ["DZJD21002362", "DZJD21002363", "DZJD21002381", "DZJD21002394","DZJD21002407"]

In [ ]:
lists = [r"\@", r"\$", r"\%", r"\#", r"\&", r"\*", r"\(", r"\)", r"\_", r"\-", r"\+", r"\=", r"\:", r"\;", r"\~", r"\\", r"\|", r"\/", r"\[", r"\]", r"\{", r"\}", r"\`", r"\>", r"\<"]

for list in lists:
  try:
    # df_la = df_la[~df_la["standard_form"].str.contains(list, na=True)]
    df_la = df_la[~df_la["dialect_form"].str.contains(list, na=True)]
  except:
    print(list)

In [ ]:
df_la

,Unnamed: 0,name,start,end,standard_form,dialect_form,dura
2,2,DZJD21002394,9.25,12.100000,시작하세요 대화 먼저,시작하세요 대화 먼저,2.850000
3,3,DZJD21002394,12.75,20.170000,어 편하게 역사 그러면 하자 뭐 역사 어쨌든 얘기했던 거니까.,어 편하게 역사 그러면 하주 뭐 역사 어쨌든 얘기했던 거니까.,7.420000
4,4,DZJD21002394,20.75,21.830000,뭐하지,뭐하지,1.080000
5,5,DZJD21002394,22.50,29.510000,아 제주도의 제일 전통적인 역사가뭐 있는지 알아?,아 제주도의 제일 전통적인 역사가뭐 이신줄 알맨?,7.010000
7,7,DZJD21002394,37.00,37.852632,삼별초?,삼별초?,0.852632
...,...,...,...,...,...,...,...
2255,2255,DZJD21002363,1683.84,1684.830000,비싸더라,비싸더라,0.990000
2256,2256,DZJD21002363,1685.25,1686.870000,그런 점이 싫어졌어,그런 점이 싫어졌어,1.620000
2257,2257,DZJD21002363,1689.50,1692.470000,집에만 있으면 나오는게 그거 였는데,집에만 있으면 나오는게 그거 여신디,2.970000
2258,2258,DZJD21002363,1695.00,1696.890000,집에 가도 안 나와요,집에 가도 안 나와요,1.890000


In [ ]:
audio = [] # 음성 벡터화
inst = [] # 제주방언
for i in file_name:

  temp = df_la[df_la['name'] == i]
  word = temp['dialect_form']
  start = temp['start']
  end = temp['end']
  dur = end - start
  midi_file = f'/content/jeju_stand/{i}.wav'

  for j, k, l in tqdm(zip(start, dur, word)):

    y, sr = librosa.load(midi_file, sr = None, offset = j, duration = k)
    audio.append(y)
    inst.append(l)
    # display(dur)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
len(audio)

1654

In [ ]:
# audio = []

# for i in range(len(df_la)):
#   y, sr = librosa.load(midi_file, sr = None, offset = float(df_la["start"][i]), duration = float(df_la["dura"][i]))
#   audio.append(y)

In [ ]:
# for idx in range(0, len(audio)):
#     plt.figure(figsize=(18, 2))
#     plt.plot(audio[idx])
#     plt.ylim(-0.05, 0.05)
#     plt.show()

## 들어보기 

In [ ]:
inst[15]

'그냥 되게 좋안.'

In [ ]:
ipd.Audio(audio[15], rate=sr)

In [ ]:
df_la["audio"] = audio
df_la["inst"] = inst

In [ ]:
df_la

,Unnamed: 0,name,start,end,standard_form,dialect_form,dura,audio,inst
2,2,DZJD21002394,9.25,12.100000,시작하세요 대화 먼저,시작하세요 대화 먼저,2.850000,"[-0.00021362305, -0.00021362305, -0.0002136230...",누구 먼저 얘기해
3,3,DZJD21002394,12.75,20.170000,어 편하게 역사 그러면 하자 뭐 역사 어쨌든 얘기했던 거니까.,어 편하게 역사 그러면 하주 뭐 역사 어쨌든 얘기했던 거니까.,7.420000,"[-0.00012207031, -0.00015258789, -0.0001525878...",일단 이름 얘기하세요
4,4,DZJD21002394,20.75,21.830000,뭐하지,뭐하지,1.080000,"[-0.00012207031, -0.00015258789, -0.0001525878...",나부터 얘기해
5,5,DZJD21002394,22.50,29.510000,아 제주도의 제일 전통적인 역사가뭐 있는지 알아?,아 제주도의 제일 전통적인 역사가뭐 이신줄 알맨?,7.010000,"[-0.00012207031, -0.00015258789, -0.0001831054...",여행
7,7,DZJD21002394,37.00,37.852632,삼별초?,삼별초?,0.852632,"[-0.004211426, -0.0043945312, -0.0045166016, -...",너
...,...,...,...,...,...,...,...,...,...
2255,2255,DZJD21002363,1683.84,1684.830000,비싸더라,비싸더라,0.990000,"[-0.00015258789, -0.00024414062, -0.0002136230...",그래?
2256,2256,DZJD21002363,1685.25,1686.870000,그런 점이 싫어졌어,그런 점이 싫어졌어,1.620000,"[-0.00024414062, -0.00015258789, -0.0002136230...",그리고 요즘 뭐냐 루미큐브도 루미큐브 트래블이라고 해서 여행용으로 다니고 다닐 수도...
2257,2257,DZJD21002363,1689.50,1692.470000,집에만 있으면 나오는게 그거 였는데,집에만 있으면 나오는게 그거 여신디,2.970000,"[-0.00024414062, -0.00021362305, -0.0003051757...",어 맞아 나 좀 쬐끄만한디
2258,2258,DZJD21002363,1695.00,1696.890000,집에 가도 안 나와요,집에 가도 안 나와요,1.890000,"[-0.0140686035, -0.010772705, -0.009613037, -0...",네가 그거인 거 닮.


# 220916 시도

### vocab

In [ ]:
vocab=['가','각','갂','간','갇','갈','갉',
    '감','갑','값','갓','갔','강','갖','갗','같','갚','갛','개','객','갠','갤','갬','갭','갯','갰','갱',
    '갸','갹','걀','걔','거','걱','건','걷','걸','걹','검','겁','것','겄','겅','겆','겉','겊','겋','게',
    '겐','겔','겜','겟','겠','겡','겨','격','겪','견','겯','결','겸','겹','겼','경','곁','계','곗','고',
    '곡','곤','곧','골','곪','곬','곯','곰','곱','곳','공','곶','곺','과','곽','관','괄','괌','괍','괏',
    '광','괘','괙','괜','괠','괨','괩','괭','괴','괸','괼','굄','굉','교','구','국','군','굳','굴','굵',
    '굶','굼','굽','굿','궁','궂','궈','권','궐','궤','귀','귄','귈','귐','귑','귓','규','균','귤','귬',
    '그','극','근','귿','글','긁','금','급','긋','긍','기','긱','긴','긷','길','김','깁','깃','깄','깅',
    '깊','까','깍','깎','깐','깔','깜','깝','깠','깡','깢','깥','깨','깩','깬','깰','깸','깹','깻','깼',
    '깽','꺄','꺅','꺼','꺽','꺾','껀','껄','껌','껍','껏','껐','껑','께','껜','껴','꼍','꼬','꼭','꼰',
    '꼴','꼼','꼽','꼿','꽁','꽂','꽃','꽈','꽉','꽌','꽘','꽜','꽝','꽤','꽥','꽹','꾀','꾄','꾈','꾐',
    '꾕','꾜','꾸','꾹','꾼','꿀','꿇','꿈','꿉','꿋','꿍','꿎','꿔','꿘','꿩','꿰','뀀','뀌','뀍','뀐',
    '뀔','뀜','뀝','뀨','끄','끅','끈','끊','끌','끓','끔','끕','끗','끙','끝','끼','끽','낀','낄','낌',
    '낍','낏','낐','낑','나','낙','낚','난','낟','날','낡','남','납','낫','났','낭','낮','낯','낱','낳',
    '내','낵','낸','낻','낼','냄','냅','냇','냈','냉','냐','냑','냠','냥','냬','너','넉','넋','넌','넏',
    '널','넓','넘','넙','넛','넜','넝','넞','넣','네','넥','넨','넬','넴','넵','넷','녀','녁','년','념',
    '녑','녓','녔','녕','녘','녜','노','녹','녺','논','놀','놈','놉','놋','농','높','놓','놔','놨','뇌',
    '뇐','뇔','뇜','뇨','뇽','누','눅','눈','눋','눌','눓','눔','눕','눗','눠','눳','눴','뉘','뉜','뉠',
    '뉨','뉩','뉴','늄','늉','느','늑','는','늘','늙','늠','늡','능','늦','늪','늬','니','닉','닌','닐',
    '님','닙','닛','닝','닢','다','닥','닦','단','닫','달','닭','닮','닳','담','답','닷','당','닺','닻',
    '닿','대','댁','댄','댈','댐','댑','댓','댔','댕','더','덕','던','덛','덜','덟','덤','덥','덧','덨',
    '덩','덪','덫','덮','데','덱','덴','델','뎀','뎃','뎅','뎌','도','독','돈','돋','돌','돐','돔','돕',
    '돗','동','돛','돼','됀','됐','되','된','될','됨','됩','됫','됬','두','둑','둔','둘','둠','둡','둣',
    '둥','둬','뒀','뒈','뒤','뒷','듀','듐','드','득','든','듣','들','듬','듭','듯','등','디','딕','딘',
    '딛','딜','딤','딥','딧','딨','딩','딪','따','딱','딲','딴','딷','딸','땀','땁','땃','땄','땅','땋',
    '때','땍','땐','땔','땜','땝','땟','땠','땡','떠','떡','떤','떨','떫','떰','떱','떳','떴','떵','떻',
    '떼','뗀','뗄','뗌','뗏','뗑','또','똑','똔','똘','똥','뙤','뚜','뚝','뚤','뚫','뚬','뚱','뛰','뛴',
    '뛸','뜀','뜁','뜨','뜩','뜬','뜯','뜰','뜸','뜹','뜻','띄','띈','띠','띤','띨','띰','띱','띵','라',
    '락','란','랄','람','랍','랏','랐','랑','랗','래','랙','랜','랠','램','랩','랫','랬','랭','랴','략',
    '량','러','럭','런','럴','럼','럽','럿','렀','렁','렇','레','렉','렌','렐','렘','렙','렛','렝','려',
    '력','련','렬','렴','렵','렷','렸','령','례','로','록','론','롤','롬','롭','롯','롱','뢰','료','룡',
    '루','룩','룬','룰','룸','룹','룻','룽','뤄','뤼','류','륙','륜','률','륨','륭','르','륵','른','를',
    '름','릅','릇','릉','릎','리','릭','린','릴','림','립','릿','링','마','막','만','많','맏','말','맑',
    '맘','맙','맛','망','맞','맡','맣','매','맥','맨','맬','맴','맵','맷','맸','맹','맺','먀','머','먹',
    '먼','멀','멈','멉','멋','멍','멎','멓','메','멕','멘','멜','멤','멥','멧','며','멱','면','멸','몃',
    '몄','명','몇','모','목','몫','몬','몰','몸','몹','못','몽','뫃','뫼','묏','묘','무','묵','묶','문',
    '묻','물','묽','뭄','뭇','뭉','뭍','뭏','뭐','뭔','뭘','뭡','뭣','뮈','뮌','뮤','므','믄','믈','믐',
    '미','믹','민','믿','밀','밈','밉','밋','밍','및','밑','뮴','바','박','밖','반','받','발','밝','밟',
    '밤','밥','밧','방','밭','배','백','밴','밷','밸','뱀','뱁','뱃','뱄','뱅','버','벅','번','벋','벌',
    '범','법','벗','벘','벙','벚','베','벡','벤','벧','벨','벰','벱','벳','벵','벼','벽','변','별','볌',
    '볍','볏','볐','병','볒','볕','보','복','볶','본','볼','봄','봅','봇','봉','봐','봔','봤','뵈','뵉',
    '뵌','뵐','뵘','뵙','뵤','부','북','분','붇','불','붉','붐','붑','붓','붕','붙','붜','뷔','뷰','브',
    '븍','븐','블','븜','븝','븟','비','빅','빈','빋','빌','빎','빔','빕','빗','빙','빚','빛','빠','빡',
    '빤','빨','빰','빱','빳','빴','빵','빻','빼','빽','뺀','뺄','뺌','뺏','뺐','뺑','뺘','뺨','뻐','뻑',
    '뻔','뻗','뻘','뻠','뻣','뻤','뻥','뻬','뼈','뼉','뼘','뼝','뽀','뽁','뽄','뽈','뽐','뽑','뽕','뾰',
    '뿀','뿅','뿌','뿍','뿐','뿔','뿜','뿝','뿡','쁘','쁙','쁜','쁠','쁨','쁩','삐','삑','삔','삘','삠',
    '삣','삥','사','삭','삮','삯','산','삳','살','삵','삶','삼','삽','삿','샀','상','샅','새','색','샌',
    '샏','샐','샘','샙','샛','샜','생','샤','샥','샨','샬','샴','샷','샹','서','석','섞','선','섣','설',
    '섥','섦','섧','섪','섬','섭','섯','섰','성','섶','세','섹','센','셀','셈','셉','셋','셍','셑','셔',
    '션','셧','셨','셰','소','속','솎','손','솔','솜','솝','솟','송','솥','솨','솩','솰','쇄','쇈','쇗',
    '쇠','쇤','쇨','쇰','쇱','쇳','쇼','쇽','숀','숄','숍','수','숙','순','숟','술','숨','숩','숫','숭',
    '숯','숱','숲','숴','쉐','쉘','쉬','쉰','쉴','쉼','쉽','쉿','슁','슈','슉','슐','슘','슛','슝','스',
    '슥','슨','슬','슭','슴','습','슷','승','시','식','신','싣','실','싫','심','십','싯','싱','싶','싸',
    '싹','싼','쌀','쌈','쌉','쌋','쌌','쌍','쌓','쌔','쌕','쌘','쌜','쌤','쌧','쌨','쌩','쌰','썃','썅',
    '써','썩','썬','썰','썲','썸','썹','썼','썽','쎄','쎈','쎙','쏘','쏙','쏜','쏟','쏠','쏨','쏭','쏴',
    '쏵','쐐','쐬','쐰','쐴','쐼','쐿','쑤','쑥','쑨','쑬','쑴','쑹','쒀','쒔','쓔','쓕','쓰','쓱','쓴',
    '쓸','씀','씁','씌','씐','씨','씩','씬','씰','씸','씹','씻','씽','아','악','안','앉','않','알','앎',
    '앓','암','압','앗','았','앙','앞','앟','애','액','앤','앨','앰','앱','앳','앴','앵','야','약','얀',
    '얄','얇','얌','얍','얏','얐','양','얕','얗','얘','얜','어','억','얶','언','얹','얺','얻','얼','얽',
    '엄','업','없','엇','었','엉','엊','엌','엎','에','엑','엔','엘','엠','엡','엣','엥','여','역','엮',
    '연','열','엶','엷','염','엽','엾','엿','였','영','옆','옇','예','옌','옐','옛','오','옥','온','올',
    '옭','옮','옳','옴','옵','옷','옹','옺','옻','와','왁','완','왈','왐','왑','왓','왔','왜','왝','왠',
    '왱','외','왹','왼','욀','욈','욋','욍','요','욕','욘','욥','욧','용','우','욱','운','욷','울','욹',
    '욺','움','웁','웃','웅','워','웍','원','월','웜','웝','웟','웠','웡','웨','웩','웬','웰','웸','위',
    '윅','윈','윌','윔','윗','윙','유','육','윤','율','윰','융','윷','으','윽','은','을','읊','음','읍',
    '응','의','이','익','인','일','읽','잃','임','입','잇','있','잉','잊','잎','왕','자','작','잔','잖',
    '잘','잠','잡','잣','잤','장','잦','잧','재','잭','잰','잴','잼','잽','잿','쟀','쟁','쟈','쟉','쟌',
    '쟘','저','적','전','절','젊','점','접','젓','젔','정','젖','제','젝','젠','젤','젬','젭','젯','져',
    '젼','졌','조','족','존','졸','졺','좀','좁','종','좆','좇','좋','좌','좍','좐','좔','좜','좝','좟',
    '좠','좨','좬','좰','좸','죄','죈','죌','죔','죕','죗','죠','죤','죰','죱','죵','주','죽','준','줄',
    '줆','줌','줍','줏','중','줘','줬','쥐','쥔','쥘','쥠','쥡','쥬','쥰','쥴','즈','즉','즌','즐','즘',
    '즙','증','지','직','진','짇','질','짊','짐','집','짓','징','짖','짙','짚','짜','짝','짠','짢','짤',
    '짧','짬','짭','짯','짰','짱','째','짹','짼','쨀','쨈','쨉','쨋','쨌','쨍','쨘','쩌','쩍','쩐','쩔',
    '쩜','쩝','쩡','쪄','쪘','쪼','쪽','쫀','쫄','쫌','쫍','쫑','쫒','쫓','쫘','쫙','쬐','쬔','쬘','쬠',
    '쬡','쭁','쭈','쭉','쭐','쭘','쭙','쭝','쭤','쮸','쯔','쯤','쯧','찌','찍','찐','찔','찜','찝','찡',
    '찢','찧','차','착','찬','찮','찯','찰','참','찹','찻','찼','창','찾','찿','채','책','챈','챌','챔',
    '챕','챘','챙','챠','챤','챰','챱','처','척','천','철','첨','첩','첫','청','체','첵','첸','첼','쳇',
    '쳐','쳤','초','촉','촌','촐','촘','촙','촛','총','촬','최','쵸','추','축','춘','출','춤','춥','춧',
    '충','춰','췄','췌','취','츄','츠','측','츰','층','치','칙','친','칠','칡','침','칩','칫','칭','카',
    '칵','칸','칼','캄','캅','캇','캉','캐','캑','캔','캘','캠','캡','캣','캥','캬','캭','커','컨','컬',
    '컴','컵','컷','컹','케','켁','켄','켈','켐','켑','켓','켕','켜','켠','켤','켬','켭','켯','켰','코',
    '콕','콘','콜','콤','콥','콧','콩','콰','콱','콴','콸','쾀','쾅','쾌','쾍','쾡','쾰','쿄','쿠','쿡',
    '쿤','쿨','쿰','쿳','쿵','쿼','퀘','퀭','퀴','퀵','퀸','큐','크','큰','클','큼','큽','킁','키','킥',
    '킨','킬','킴','킵','킷','킹','타','탁','탄','탈','탉','탐','탑','탓','탔','탕','태','택','탠','탤',
    '탬','탭','탯','탰','탱','터','턱','턴','털','텀','텁','텃','텄','텅','테','텍','텐','텔','템','텝',
    '텟','텠','텡','텨','토','톡','톤','톨','톰','톱','톳','통','퇘','퇴','투','툭','툰','툴','툼','툽',
    '툿','퉁','퉤','퉷','튀','튄','튈','튐','튑','튓','튕','튜','튬','트','특','튼','튿','틀','틈','틉',
    '틋','틔','퇸','티','틱','틴','틸','팀','팁','팅','파','팍','팎','판','팔','팜','팝','팟','팠','팡',
    '팥','패','팩','팬','팰','팸','팹','팻','팼','팽','퍼','퍽','펀','펄','펌','펍','펏','펐','펑','페',
    '펙','펜','펠','펨','펩','펫','펭','펴','펵','편','펼','폄','폅','폈','평','폐','포','폭','폰','폴',
    '폼','폽','폿','퐁','푀','푄','표','푯','푸','푹','푼','풀','품','풉','풋','풍','퓌','퓨','프','픈',
    '플','픔','픕','피','픽','핀','필','핌','핍','핏','핑','하','학','한','할','핥','함','합','핫','핬',
    '항','핱','해','핵','핸','핼','햄','햅','햇','했','행','햐','향','허','헉','헌','헐','헒','험','헙',
    '헛','헝','헤','헥','헨','헬','헴','헵','헷','헹','혀','혁','현','혈','혐','협','혓','혔','형','혜',
    '호','혹','혼','홀','홈','홉','홋','홍','화','확','환','활','홧','황','홰','홱','횃','회','획','횟',
    '횡','효','후','훅','훈','훌','훑','훔','훗','훠','훤','훨','훰','훼','휀','휑','휘','휙','휜','휠',
    '휨','휩','휭','휴','휼','흄','흉','흐','흑','흔','흗','흘','흙','흠','흡','흣','흥','흩','희','흰',
    '흴','히','힉','힌','힐','힘','힙','힛','힝','괵','굻','궉','궝','궹','귕','긇','긏','긑',
    '긔','깟','껭','꼉','꽐','꾿','끠','맄','앍','앏','앒','얫','얱','옜','옝','웽','윳','쭌']

### try!

In [ ]:
alp = [x for x in "abcdefghijklmnopqrstuvwxyz"]

lst = ['&', '#', '\@', '\$', '%', '\*', '\(', '\)', '\+', '_',
       ':', ';', '\^', '-', '_', '=', '/', r'\\', '\|',
       '~', '>', '<', '\[', '\]', '\{', '\}', '`'] + alp

df_sound = df_sound.dropna(axis=0)

for i in lst:

  df_sound = df_sound[~df_sound['dialect_form'].str.contains(i, case=False)]

df_sound.reset_index(drop=True)

In [ ]:
# 대본에 사용할 수 있는 문자 집합입니다.
characters = vocab
print('acceptable characters : ', characters)

# characters -> 정수 
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")

# 정수 -> characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

acceptable characters :  ['가', '각', '갂', '간', '갇', '갈', '갉', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '같', '갚', '갛', '개', '객', '갠', '갤', '갬', '갭', '갯', '갰', '갱', '갸', '갹', '걀', '걔', '거', '걱', '건', '걷', '걸', '걹', '검', '겁', '것', '겄', '겅', '겆', '겉', '겊', '겋', '게', '겐', '겔', '겜', '겟', '겠', '겡', '겨', '격', '겪', '견', '겯', '결', '겸', '겹', '겼', '경', '곁', '계', '곗', '고', '곡', '곤', '곧', '골', '곪', '곬', '곯', '곰', '곱', '곳', '공', '곶', '곺', '과', '곽', '관', '괄', '괌', '괍', '괏', '광', '괘', '괙', '괜', '괠', '괨', '괩', '괭', '괴', '괸', '괼', '굄', '굉', '교', '구', '국', '군', '굳', '굴', '굵', '굶', '굼', '굽', '굿', '궁', '궂', '궈', '권', '궐', '궤', '귀', '귄', '귈', '귐', '귑', '귓', '규', '균', '귤', '귬', '그', '극', '근', '귿', '글', '긁', '금', '급', '긋', '긍', '기', '긱', '긴', '긷', '길', '김', '깁', '깃', '깄', '깅', '깊', '까', '깍', '깎', '깐', '깔', '깜', '깝', '깠', '깡', '깢', '깥', '깨', '깩', '깬', '깰', '깸', '깹', '깻', '깼', '깽', '꺄', '꺅', '꺼', '꺽', '꺾', '껀', '껄', '껌', '껍', '껏', '껐', '껑', '께', '껜', '껴', '꼍', '꼬', '꼭', '꼰', '꼴', '꼼', '꼽', '꼿', '꽁', '꽂', '꽃', '꽈',

In [ ]:
# An integer scalar Tensor. The window length in samples.
frame_length = 256

# An integer scalar Tensor. The number of samples to step.
frame_step = 160

# An integer scalar Tensor. The size of the FFT to apply.
# If not provided, uses the smallest power of 2 enclosing frame_length.
fft_length = 384


def encode_single_sample(wav_file, label):

    ###########################################
    ##  Process the Audio
    ##########################################
    # 1. Read wav file
    # file_ = tf.io.read_file(wavs_path + wav_file + ".wav")
    # 2. Decode the wav file
    audio2, _ = tf.audio.decode_wav(wav_file) # wav -> float tensor 변환
    audio2 = tf.squeeze(audio2, axis=-1)
    # 3. Change type to float
    audio = tf.cast(audio2, tf.float32)
    # 4. Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )

    # 5. We only need the magnitude, which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)

    # 6. normalisation
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True) # 특정 차원을 제거하고 평균
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True) # 특정 차원을 제거하고 표준편차
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)


    ###########################################
    ##  Process the label
    ##########################################

    # 8. Split the label
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # 9. Map the characters in label to numbers
    label = char_to_num(label)

    # 10. Return a dict as our model is expecting two inputs
    return spectrogram, label


In [ ]:
batch_size = 32
# Define the trainig dataset
for i in range(len(df_la)):
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (list(df_la["audio"]), list(df_la["inst"]))
  )
  train_dataset = (
      train_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
      .padded_batch(batch_size)
      .prefetch(buffer_size=tf.data.AUTOTUNE)
  )

# # Define the validation dataset
# validation_dataset = tf.data.Dataset.from_tensor_slices(
#     (list(df_la["file_name"]), list(df_la["normalized_transcription"]))
# )
# validation_dataset = (
#     validation_dataset.map(encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE)
#     .padded_batch(batch_size)
#     .prefetch(buffer_size=tf.data.AUTOTUNE)
# )


TypeError: ignored

In [ ]:
fig = plt.figure(figsize=(8, 5))

for batch in train_dataset.take(1):
    spectrogram = batch[0][0].numpy()
    spectrogram = np.array([np.trim_zeros(x) for x in np.transpose(spectrogram)])
    label = batch[1][0]

    # Spectrogram
    label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
    ax = plt.subplot(2, 1, 1)
    ax.imshow(spectrogram, vmax=1)
    ax.set_title(label)
    ax.axis("off")

    # Wav
    file_ = tf.io.read_file(wavs_path + list(df_train["file_name"])[0] + ".wav")
    audio, _ = tf.audio.decode_wav(file_)
    audio = audio.numpy()
    ax = plt.subplot(2, 1, 2)
    plt.plot(audio)
    ax.set_title("Signal Wave")
    ax.set_xlim(0, len(audio))
    display.display(display.Audio(np.transpose(audio), rate=22050))

plt.show()